<a href="https://colab.research.google.com/github/lalith-kumar-raju/Infosys-Springboard-6.0/blob/main/SQL_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sqlite3
import pandas as pd

In [3]:
# Create a new in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

In [4]:
# Step 1: Create a small sample table
cursor.execute("""
CREATE TABLE Malware (
    id INTEGER,
    feature1 REAL,
    feature2 REAL,
    feature3 TEXT,
    label TEXT
)
""")

In [5]:
# Step 2: Insert some sample data
data = [
    (1, 120.5, 200.1, 'X1', 'Malicious'),
    (2, 85.3, 150.6, 'X2', 'Benign'),
    (3, 95.7, 180.2, 'X1', 'Malicious'),
    (4, None, 210.0, 'X3', 'Malicious'),
    (5, 60.2, None, 'X2', 'Benign'),
    (6, 120.5, 200.1, 'X1', 'Malicious')  # duplicate row
]
cursor.executemany("INSERT INTO Malware VALUES (?, ?, ?, ?, ?)", data)
conn.commit()

# Helper function to run query and show result
def run_query(query):
    return pd.read_sql_query(query, conn)

In [6]:
# Step 3: Perform the SQL operations

# 1. Shape (row count)
print("1. Shape:")
print(run_query("SELECT COUNT(*) AS total_rows FROM Malware"))

1. Shape:
   total_rows
0           6


In [7]:
# 2. Column names
print("\n2. Column names:")
print(run_query("PRAGMA table_info(Malware)"))


2. Column names:
   cid      name     type  notnull dflt_value  pk
0    0        id  INTEGER        0       None   0
1    1  feature1     REAL        0       None   0
2    2  feature2     REAL        0       None   0
3    3  feature3     TEXT        0       None   0
4    4     label     TEXT        0       None   0


In [8]:
# 3. Data types
print("\n3. Data types:")
print(run_query("PRAGMA table_info(Malware)"))


3. Data types:
   cid      name     type  notnull dflt_value  pk
0    0        id  INTEGER        0       None   0
1    1  feature1     REAL        0       None   0
2    2  feature2     REAL        0       None   0
3    3  feature3     TEXT        0       None   0
4    4     label     TEXT        0       None   0


In [9]:
# 4. Missing values
print("\n4. Missing values:")
print(run_query("""
SELECT
    SUM(CASE WHEN feature1 IS NULL THEN 1 ELSE 0 END) AS missing_feature1,
    SUM(CASE WHEN feature2 IS NULL THEN 1 ELSE 0 END) AS missing_feature2,
    SUM(CASE WHEN feature3 IS NULL THEN 1 ELSE 0 END) AS missing_feature3,
    SUM(CASE WHEN label IS NULL THEN 1 ELSE 0 END) AS missing_label
FROM Malware
"""))


4. Missing values:
   missing_feature1  missing_feature2  missing_feature3  missing_label
0                 1                 1                 0              0


In [10]:
# 5. Summary statistics
print("\n5. Summary statistics:")
print(run_query("SELECT AVG(feature1) AS avg_feature1, MIN(feature1) AS min_feature1, MAX(feature1) AS max_feature1 FROM Malware"))


5. Summary statistics:
   avg_feature1  min_feature1  max_feature1
0         96.44          60.2         120.5


In [11]:
# 6. Value counts
print("\n6. Value counts (labels):")
print(run_query("SELECT label, COUNT(*) AS freq FROM Malware GROUP BY label"))


6. Value counts (labels):
       label  freq
0     Benign     2
1  Malicious     4


In [12]:
# 7. Filter rows
print("\n7. Filter rows (Malicious only):")
print(run_query("SELECT * FROM Malware WHERE label = 'Malicious'"))


7. Filter rows (Malicious only):
   id  feature1  feature2 feature3      label
0   1     120.5     200.1       X1  Malicious
1   3      95.7     180.2       X1  Malicious
2   4       NaN     210.0       X3  Malicious
3   6     120.5     200.1       X1  Malicious


In [13]:
# 8. Sort values
print("\n8. Sort by feature1:")
print(run_query("SELECT * FROM Malware ORDER BY feature1 DESC"))


8. Sort by feature1:
   id  feature1  feature2 feature3      label
0   1     120.5     200.1       X1  Malicious
1   6     120.5     200.1       X1  Malicious
2   3      95.7     180.2       X1  Malicious
3   2      85.3     150.6       X2     Benign
4   5      60.2       NaN       X2     Benign
5   4       NaN     210.0       X3  Malicious


In [14]:
# 9. Unique values
print("\n9. Unique labels:")
print(run_query("SELECT DISTINCT label FROM Malware"))


9. Unique labels:
       label
0  Malicious
1     Benign


In [15]:
# 10. Group by and count
print("\n10. Group by label (count):")
print(run_query("SELECT label, COUNT(*) AS cnt FROM Malware GROUP BY label"))


10. Group by label (count):
       label  cnt
0     Benign    2
1  Malicious    4


In [16]:
# 11. Group by and average
print("\n11. Group by label (avg feature1):")
print(run_query("SELECT label, AVG(feature1) AS avg_feature1 FROM Malware GROUP BY label"))


11. Group by label (avg feature1):
       label  avg_feature1
0     Benign     72.750000
1  Malicious    112.233333


In [17]:
# 12. Top N rows
print("\n12. Top 3 rows:")
print(run_query("SELECT * FROM Malware LIMIT 3"))


12. Top 3 rows:
   id  feature1  feature2 feature3      label
0   1     120.5     200.1       X1  Malicious
1   2      85.3     150.6       X2     Benign
2   3      95.7     180.2       X1  Malicious


In [18]:
# 13. Check duplicates
print("\n13. Duplicates:")
print(run_query("SELECT id, COUNT(*) AS cnt FROM Malware GROUP BY id HAVING COUNT(*) > 1"))


13. Duplicates:
Empty DataFrame
Columns: [id, cnt]
Index: []


In [19]:
# 14. Remove duplicates (create new table)
cursor.execute("CREATE TABLE Malware_no_duplicates AS SELECT DISTINCT * FROM Malware")
print("\n14. Malware_no_duplicates table created.")


14. Malware_no_duplicates table created.


In [20]:
# 15. Correlation (SQLite doesn’t support corr, so we approximate with covariance formula if needed)
print("\n15. Correlation (manual calc not supported in SQLite). Skipped.")


15. Correlation (manual calc not supported in SQLite). Skipped.


In [21]:
# 16. Replace NULL
print("\n16. Replace NULL in feature3:")
print(run_query("SELECT id, COALESCE(feature3, 'Unknown') AS updated_feature3 FROM Malware"))


16. Replace NULL in feature3:
   id updated_feature3
0   1               X1
1   2               X2
2   3               X1
3   4               X3
4   5               X2
5   6               X1


In [22]:
# 17. Join example (create second table)
cursor.execute("""
CREATE TABLE Details (
    malware_id INTEGER,
    extra_info TEXT
)
""")
cursor.executemany("INSERT INTO Details VALUES (?, ?)", [(1, "High risk"), (2, "Low risk"), (3, "Medium risk")])
conn.commit()

print("\n17. Join with Details:")
print(run_query("SELECT m.id, m.label, d.extra_info FROM Malware m LEFT JOIN Details d ON m.id = d.malware_id"))


17. Join with Details:
   id      label   extra_info
0   1  Malicious    High risk
1   2     Benign     Low risk
2   3  Malicious  Medium risk
3   4  Malicious         None
4   5     Benign         None
5   6  Malicious         None


In [23]:
# 18. Aggregate multiple stats
print("\n18. Multiple aggregates:")
print(run_query("SELECT AVG(feature1) AS avg_f1, SUM(feature2) AS sum_f2, COUNT(*) AS row_count FROM Malware"))


18. Multiple aggregates:
   avg_f1  sum_f2  row_count
0   96.44   941.0          6


In [24]:
# 19. Conditional column
print("\n19. Conditional (High/Low):")
print(run_query("SELECT feature1, CASE WHEN feature1 > 100 THEN 'High' ELSE 'Low' END AS f1_category FROM Malware"))


19. Conditional (High/Low):
   feature1 f1_category
0     120.5        High
1      85.3         Low
2      95.7         Low
3       NaN         Low
4      60.2         Low
5     120.5        High


In [25]:
# 20. Delete rows with NULLs
cursor.execute("DELETE FROM Malware WHERE feature1 IS NULL OR feature2 IS NULL")
print("\n20. Rows with NULLs deleted.")
print(run_query("SELECT * FROM Malware"))



20. Rows with NULLs deleted.
   id  feature1  feature2 feature3      label
0   1     120.5     200.1       X1  Malicious
1   2      85.3     150.6       X2     Benign
2   3      95.7     180.2       X1  Malicious
3   6     120.5     200.1       X1  Malicious
